# Custom Unit Example

This example demonstrates how to create energy units with custom behavior. We include a simple implementation of a stochastic unit which samples outages sequentially based on mean-time-to-repair and mean-time-to-failure rather than independently based on forced outage rates. 

**Note:** We do not claim the correctness of this implementation. It is neither well-tested nor researched, rather it demonstrates the necessary modifications researchers can make to extend the functionality of `assetra`.

To create our `SequentialStochasticUnit` class, we must implement the three abstract methods defined in the `EnergyUnit` base class.

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime
from random import gauss

import xarray as xr

from assetra.units import EnergyUnit

@dataclass(frozen=True)
class SequentialStochasticUnit(EnergyUnit):
    hourly_capacity: xr.DataArray
    mean_time_to_failure: float
    stdev_time_to_failure: float
    mean_time_to_repair: float
    stdev_time_to_repair: float

    def get_hourly_capacity_trial(self, time:slice):
        hourly_capacity_slice = self.hourly_capacity.sel(time=time)
        hourly_unit_state = []
        unit_state = True

        # find first transition
        next_transition_hour = max(0, int(gauss(
            self.mean_time_to_failure,
            self.stdev_time_to_failure
        )))

        # find states
        while len(hourly_unit_state) < len(hourly_capacity_slice):
            hour = len(hourly_unit_state)
            if hour == next_transition_hour:
                # if this hour is a transition hour
                # change states
                unit_state = not unit_state

                # find next transition
                if unit_state:
                    # if unit is on, find next failure
                    next_transition_hour = hour + max(1, int(gauss(
                        self.mean_time_to_failure,
                        self.stdev_time_to_failure
                    )))
                else: 
                    # otherwise, find next repair
                    next_transition_hour = hour + max(1, int(gauss(
                        self.mean_time_to_repair,
                        self.stdev_time_to_repair
                    )))
            hourly_unit_state.append(unit_state)
        return hourly_capacity_slice.where(hourly_unit_state, 0)

    @staticmethod
    def to_unit_dataset(units: list[SequentialStochasticUnit]) -> xr.Dataset:
        """Convert a list of energy units of the derived class type into an
        xarray dataset.

        For different energy units, different dataset
        dimensions and coordinates may be appropriate.

        Args:
            units (list[EnergyUnit]): List of of energy units of the derived
                class type. 

        Returns:
            xr.Dataset: Dataset storing sufficient information to (1) fully
                reconstruct the list of energy units from which it is created
                and (2) generate hourly capacity time series with the
                EnergyUnit.get_probabilistic_capacity_matrix function
        """
        unit_dataset = xr.Dataset(
            data_vars=dict(
                nameplate_capacity=(
                    ["energy_unit"],
                    [unit.nameplate_capacity for unit in units],
                ),
                hourly_capacity=(
                    ["energy_unit", "time"],
                    [unit.hourly_capacity for unit in units],
                ),
                mean_time_to_failure=(
                    ["energy_unit"],
                    [unit.mean_time_to_failure for unit in units],
                ),
                stdev_time_to_failure=(
                    ["energy_unit"],
                    [unit.stdev_time_to_failure for unit in units],
                ),
                mean_time_to_repair=(
                    ["energy_unit"],
                    [unit.mean_time_to_repair for unit in units],
                ),
                stdev_time_to_repair=(
                    ["energy_unit"],
                    [unit.stdev_time_to_repair for unit in units],
                ),
            ),
            coords=dict(
                energy_unit=[unit.id for unit in units],
                time=units[0].hourly_capacity.time if len(units) > 0 else [],
            ),
        )

        return unit_dataset

    @staticmethod
    def from_unit_dataset(unit_dataset: xr.Dataset) -> list[SequentialStochasticUnit]:
        """Convert a unit dataset to a list of energy units of the derived
        energy unit type.

        This is the inverse to the derived SequentialStochasticUnit.to_unit_dataset function

        Args:
            unit_dataset (xr.Dataset): Unit dataset with structure and content
                defined in the derived EnergyUnit.to_unit_dataset function

        Returns:
            list[EnergyUnit]: List of energy units of the derived class type
        """
        # build list
        units = []

        for id in unit_dataset.energy_unit:
            units.append(
                SequentialStochasticUnit(
                    int(id),
                    int(unit_dataset.nameplate_capacity.loc[id]),
                    unit_dataset.hourly_capacity.loc[id],
                    float(unit_dataset.mean_time_to_failure.loc[id]),
                    float(unit_dataset.stdev_time_to_failure.loc[id]),
                    float(unit_dataset.mean_time_to_repair.loc[id]),
                    float(unit_dataset.stdev_time_to_repair.loc[id])
                )
            )

        return units

    @staticmethod
    def get_probabilistic_capacity_matrix(
        unit_dataset: xr.Dataset, net_hourly_capacity_matrix: xr.DataArray
    ) -> xr.DataArray:
        """Return probabilistic hourly capacity matrix for a fleet of energy
        units of the derived energy unit type.

        Take the unit dataset and create a matrix representing the total hourly
        capacity of all energy units for some number of monte carlo trials. The
        hours and number of trials should match the net hourly capacity matrix.

        Args:
            unit_dataset (xr.Dataset): Unit dataset for the derived energy unit
                type, e.g. generated with the derived
                EnergyUnit.to_unit_dataset function
            net_hourly_capacity_matrix (xr.DataArray): Probabilistic net hourly
                capacity matrix with dimensions (trials, time) and shape
                (# of trials, # of hours)

        Returns:
            xr.DataArray: Combined hourly capacity for all units in the unit
                dataset for a determined number of Monte Carlo trials. The
                dimensions and coordinates of this matrix should match the net
                hourly capacity matrix
        """
        hourly_capacity_matrix = xr.zeros_like(net_hourly_capacity_matrix)
        units = SequentialStochasticUnit.from_unit_dataset(unit_dataset)

        for unit in units:
            for trial in hourly_capacity_matrix:
                trial += unit.get_hourly_capacity_trial(time=trial.time)

        return hourly_capacity_matrix

We are ready to use our custom class! We can begin building our energy system and declare an instance of the `SequentialStochasticUnit` type.

In [ ]:
# helper function
def get_hourly_time_series_xr(
    hourly_data: list[float], 
    start_hour: datetime="2019-01-01 00:00:00"
) -> xr.DataArray:
    '''Return formatted xarray data array for a sequence of hourly datapoints

    Args:
        hourly_data (list[float]): Input data stored as consecutive hour-scale datapoints.
        start_hour (_type_, optional): Time stamp corresponding to the first datapoint.
            Defaults to "2016-01-01 00:00:00".

    Returns:
        xr.Dataarray: Formatted one-dimensional xarray data with datetime-indexed time series.
    '''
    return xr.DataArray(
        data=[float(x) for x in hourly_data],
        coords=dict(
            time=xr.date_range(start_hour, freq='1H', periods=len(hourly_data))
        )
    )

from assetra.system import EnergySystemBuilder
from assetra.units import DemandUnit

# initialize system builder
builder = EnergySystemBuilder()

# add demand
builder.add_unit(
    DemandUnit(
        id=0,
        hourly_demand=get_hourly_time_series_xr([1]*8760)
    )
)

# create custom sequential unit
new_custom_unit = SequentialStochasticUnit(
    id=1,
    nameplate_capacity=1,
    hourly_capacity=get_hourly_time_series_xr([1]*8760),
    mean_time_to_failure=100,
    stdev_time_to_failure=20,
    mean_time_to_repair=24,
    stdev_time_to_repair=3
)

If we try to add the custom unit at this point, the system builder will throw an warning.

In [ ]:
builder.add_unit(new_custom_unit)

We need to tell `assetra` that we have created a new class. Note the following excerpt from the documentation:

> The dispatch order of unit datasets in probabilistic simulations is defined by two variables in the `assetra.units` module, specifically `RESPONSIVE_UNIT_TYPES` and `NONRESPONSIVE_UNIT_TYPES`. These two variables are lists which both define valid energy unit types and distinguish the order of unit dispatch. The responsive/non-responsive nomenclature refers to whether the hourly capacity of units of a given type depend on system conditions. For example, `StaticUnit` and `StochasticUnit` qualify as non-responsive because their probabilistic hourly capacities do not depend on the net hourly capacity matrix. `StorageUnit` on the other hand qualifies as a responsive type. Dispatch order follows the combined list (NONRESPONSIVE_UNIT_TYPES + RESPONSIVE_UNIT_TYPES).

Our new class is non-responsive (because its behavior does not depend on net hourly capacity). Once we add our unit to the corresponding list, we can add our unit as expected. Be sure not to add the unit type multiple times.

In [ ]:
from assetra.units import NONRESPONSIVE_UNIT_TYPES

# add to valid unit types
NONRESPONSIVE_UNIT_TYPES.append(SequentialStochasticUnit)

# add unit to system
builder.add_unit(new_custom_unit)

We're done! We have successfully added a custom energy unit type without modifying the `assetra` codebase. We can run our analyses as performed previously.

In [ ]:
from assetra.simulation import ProbabilisticSimulation

# build system
energy_system = builder.build()

# run simulation
simulation = ProbabilisticSimulation(
    "2019-01-01 00:00:00",
    "2019-12-31 23:00:00",
    trial_size=5
)
simulation.assign_energy_system(energy_system)
simulation.run()

# evaluate resource adequacy
from assetra.metrics import LossOfLoadHours

lolh = LossOfLoadHours(simulation).evaluate()
print("System LOLH:", lolh)